In [1]:
import folium 
import pandas as pd

In [2]:
# Load data
data_path = '../data/nashville_pd_slice.csv'
df = pd.read_csv(data_path)

In [3]:
df.head()

Event Number           Call Received  Complaint Number  Tencode  \
0  PD201701073542  10/22/2017 10:42:31 PM               NaN       43   
1  PD201500686601  06/26/2015 11:45:30 PM               NaN       93   
2  PD201801149095  11/28/2018 11:30:13 AM               NaN        3   
3  PD202100130941  02/25/2021 11:03:07 PM               NaN       96   
4  PD201500484393  05/08/2015 10:44:38 AM               NaN       96   

                        Tencode Description Tencode Suffix  \
0  WANT OFFICER FOR INVESTIGATION / ASSISTA              P   
1                         TRAFFIC VIOLATION            NaN   
2                                       NaN            NaN   
3                            BUSINESS CHECK            NaN   
4                            BUSINESS CHECK            NaN   

  Tencode Suffix Description Disposition Code Disposition Description  Block  \
0                   PROGRESS               2W                     NaN    NaN   
1                        NaN               9T                     NaN    NaN   
2                        NaN                4        ASSISTED CITIZEN  400.0   
3                        NaN                4        ASSISTED CITIZEN    NaN   
4                        NaN                4        ASSISTED CITIZEN    NaN   

  Street Name Unit Dispatched Shift Sector Zone     RPA  Latitude  Longitude  \
0         NaN            515B     B      S  311  8147.0       NaN        NaN   
1         NaN            423B     C      C  413  1133.0       NaN        NaN   
2    2ND AV N            2F15     A    NaN  NaN     NaN       NaN        NaN   
3         NaN            2P68     C      E  221  1979.0       NaN        NaN   
4         NaN            815B     A    PCW  NaN     NaN       NaN        NaN   

  Mapped Location  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN

In [4]:
df.shape

(10000, 19)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Event Number                10000 non-null  object 
 1   Call Received               10000 non-null  object 
 2   Complaint Number            851 non-null    float64
 3   Tencode                     10000 non-null  int64  
 4   Tencode Description         9745 non-null   object 
 5   Tencode Suffix              6031 non-null   object 
 6   Tencode Suffix Description  5237 non-null   object 
 7   Disposition Code            9960 non-null   object 
 8   Disposition Description     7839 non-null   object 
 9   Block                       2260 non-null   float64
 10  Street Name                 2699 non-null   object 
 11  Unit Dispatched             9309 non-null   object 
 12  Shift                       10000 non-null  object 
 13  Sector                      7773

In [6]:
df.describe()

Complaint Number       Tencode         Block           RPA  \
count      8.510000e+02  10000.000000   2260.000000   7996.000000   
mean       2.018087e+10     66.207700   2406.238938   5994.981866   
std        1.965526e+07     84.099312   2242.796943   9504.711316   
min        2.015001e+10      3.000000      0.000000      0.000000   
25%        2.016104e+10     43.000000    500.000000   1829.000000   
50%        2.018067e+10     70.000000   1750.000000   4627.000000   
75%        2.020028e+10     93.000000   3900.000000   8525.000000   
max        2.022002e+10   8000.000000  15100.000000  95040.000000   

          Latitude    Longitude  
count  1312.000000  1312.000000  
mean     35.985592   -86.351133  
std       2.441092     5.855890  
min       0.000000   -87.033000  
25%      36.087000   -86.821000  
50%      36.149000   -86.736000  
75%      36.198000   -86.675000  
max      36.382000     0.000000

In [7]:
# Remove redundant columns (eg. `Tencode` and `Tencode Description contain the same data`)
df.drop(['Tencode Description', 'Tencode Suffix Description',
         'Disposition Description', 'Mapped Location'],
        axis=1, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      10000 non-null  object 
 1   Call Received     10000 non-null  object 
 2   Complaint Number  851 non-null    float64
 3   Tencode           10000 non-null  int64  
 4   Tencode Suffix    6031 non-null   object 
 5   Disposition Code  9960 non-null   object 
 6   Block             2260 non-null   float64
 7   Street Name       2699 non-null   object 
 8   Unit Dispatched   9309 non-null   object 
 9   Shift             10000 non-null  object 
 10  Sector            7773 non-null   object 
 11  Zone              8727 non-null   object 
 12  RPA               7996 non-null   float64
 13  Latitude          1312 non-null   float64
 14  Longitude         1312 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 1.1+ MB


In [9]:
df.duplicated(subset='Event Number').sum()  # There are no duplicates in the data

0

In [16]:
location_df = df[~df['Latitude'].isna()]
location_df.drop(['Event Number', 'Complaint Number', 'Block',
                  'Street Name', 'Sector', 'Zone', 'RPA'],
                 axis=1, inplace=True)

/Users/josiah/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
center_lat = location_df['Latitude'].mean()
center_long = location_df['Longitude'].mean()

lat = location_df['Latitude'].to_numpy()
long = location_df['Longitude'].to_numpy()

In [34]:
location_df

Call Received  Tencode Tencode Suffix Disposition Code  \
11    05/15/2020 06:57:06 AM       43              P                1   
17    05/18/2019 07:15:12 PM       15              P               15   
25    11/25/2018 02:19:01 PM       45              P               11   
28    12/10/2020 08:04:33 AM       43              P                4   
37    07/15/2016 04:28:59 PM       96            NaN               12   
...                      ...      ...            ...              ...   
9928  02/17/2016 12:26:07 PM       93            NaN                9   
9934  03/29/2019 07:11:23 PM       43              P                6   
9940  04/15/2016 03:49:55 AM       43              P                4   
9990  04/28/2015 12:34:38 PM       43              P                4   
9994  07/22/2021 02:15:15 PM       43              R               14   

     Unit Dispatched Shift  Latitude  Longitude  
11              3280     A    36.146    -86.871  
17              4561     B    36.052    -86.715  
25               NaN     A    36.325    -86.873  
28              313A     A    36.108    -86.732  
37              123B     B    36.134    -86.889  
...              ...   ...       ...        ...  
9928            5F26     A    36.154    -86.833  
9934            313B     B    36.129    -86.715  
9940            321C     C    36.077    -86.725  
9990            351B     A    36.129    -86.715  
9994            327B     A    36.082    -86.660  

[1312 rows x 8 columns]

In [35]:
location_df_a = location_df[location_df['Shift']=='A']
lat_a = location_df_a['Latitude'].to_numpy()
long_a = location_df_a['Longitude'].to_numpy()

location_df_b = location_df[location_df['Shift']=='B']
lat_b = location_df_b['Latitude'].to_numpy()
long_b = location_df_b['Longitude'].to_numpy()

location_df_c = location_df[location_df['Shift']=='C']
lat_c = location_df_c['Latitude'].to_numpy()
long_c = location_df_c['Longitude'].to_numpy()

In [36]:
map_ = folium.Map(location=[center_lat, center_long], zoom_start=9)

In [37]:
for lat_, long_ in zip(lat_a, long_a):
    folium.CircleMarker(
        location=[lat_, long_],
        radius=1,
        color='red',
        fill_color='red').add_to(map_)

In [ ]:
for lat_, long_ in zip(lat_b, long_b):
    folium.CircleMarker(
        location=[lat_, long_],
        radius=1,
        color='black',
        fill_color='black').add_to(map_)

In [41]:
for lat_, long_ in zip(lat_c, long_c):
    folium.CircleMarker(
        location=[lat_, long_],
        radius=1,
        color='green',
        fill_color='green').add_to(map_)

In [42]:
map_